In [ ]:
# 📦 Import Modules
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import pandas as pd
import sqlite3

# 📄 Simulate patient data
np.random.seed(42)
patient_ids = [f'P{str(i).zfill(3)}' for i in range(1, 11)]
names = [f'Patient_{i}' for i in range(1, 11)]
genders = random.choices(['M', 'F'], k=10)
dobs = [datetime(1950, 1, 1) + timedelta(days=random.randint(0, 25000)) for _ in range(10)]

patients_df = pd.DataFrame({
    "patient_id": patient_ids,
    "name": names,
    "dob": [d.strftime('%Y-%m-%d') for d in dobs],
    "gender": genders
})

# 🧪 Simutlate Pathology test sample data
test_types = ['HbA1c', 'Glucose', 'Creatinine', 'Cholesterol']
samples = []

for pid in patient_ids:
    for _ in range(random.randint(1, 3)):
        sample_id = f"S{random.randint(1000, 9999)}"
        test = random.choice(test_types)
        value = round(random.uniform(4.0, 10.0), 1) if test != 'Creatinine' else random.randint(70, 150)
        unit = {
            'HbA1c': '%',
            'Glucose': 'mmol/L',
            'Creatinine': 'umol/L',
            'Cholesterol': 'mmol/L'
        }[test]
        date = datetime(2024, 10, 1) + timedelta(days=random.randint(0, 10))
        samples.append([sample_id, pid, test, value, unit, date])

samples_df = pd.DataFrame(samples, columns=[
    "sample_id", "patient_id", "test_type", "result_value", "unit", "collected_date"
])

# 🔄 Combine and data cleaning
merged_df = samples_df.merge(patients_df, on="patient_id")
merged_df["collected_date"] = pd.to_datetime(merged_df["collected_date"])

# 📊 Pivot Table：Avg results each test by patients
pivot_df = merged_df.pivot_table(
    index="patient_id",
    columns="test_type",
    values="result_value",
    aggfunc="mean"
).reset_index()

# 💾 export to csv）
merged_df.to_csv("pathology_merged_data.csv", index=False)
pivot_df.to_csv("pathology_pivot_summary.csv", index=False)

# 👀 show results
print("✅ Combined dat：")
print(merged_df.head())

print("\n✅ Pivot avg test result：")
print(pivot_df.head())


✅ Combined data：
  sample_id patient_id   test_type  result_value    unit collected_date  \
0     S9767       P001  Creatinine         138.0  umol/L     2024-10-02   
1     S8743       P001       HbA1c           4.8       %     2024-10-07   
2     S9220       P002  Creatinine          89.0  umol/L     2024-10-10   
3     S7393       P002       HbA1c           4.8       %     2024-10-10   
4     S6454       P002       HbA1c           5.4       %     2024-10-11   

        name         dob gender  
0  Patient_1  2014-02-04      F  
1  Patient_1  2014-02-04      F  
2  Patient_2  1987-04-13      M  
3  Patient_2  1987-04-13      M  
4  Patient_2  1987-04-13      M  

✅ Pivot Average test result：
test_type patient_id  Cholesterol  Creatinine  Glucose  HbA1c
0               P001          NaN       138.0      NaN    4.8
1               P002          NaN        89.0      NaN    5.1
2               P003          NaN         NaN      NaN    7.1
3               P004          NaN       111.0     

In [ ]:
conn = sqlite3.connect("Pathology.db")
merged_df.to_sql("samples", conn, if_exists="replace", index=False)
samples_df.to_sql("sampleraw", conn, if_exists="replace", index=False)
patients_df.to_sql("patientraw", conn, if_exists="replace", index=False)


# 1. Remove Abnormal data
print("🔍 Remove Abnormal data")
query1 = """
SELECT *
FROM samples
WHERE NOT (test_type = 'HbA1c' AND result_value > 15);
"""
display(pd.read_sql(query1, conn))


🔍 Remove Abnormal data


,sample_id,patient_id,test_type,result_value,unit,collected_date,name,dob,gender
0,S9767,P001,Creatinine,138.0,umol/L,2024-10-02 00:00:00,Patient_1,2014-02-04,F
1,S8743,P001,HbA1c,4.8,%,2024-10-07 00:00:00,Patient_1,2014-02-04,F
2,S9220,P002,Creatinine,89.0,umol/L,2024-10-10 00:00:00,Patient_2,1987-04-13,M
3,S7393,P002,HbA1c,4.8,%,2024-10-10 00:00:00,Patient_2,1987-04-13,M
4,S6454,P002,HbA1c,5.4,%,2024-10-11 00:00:00,Patient_2,1987-04-13,M
5,S4902,P003,HbA1c,7.1,%,2024-10-11 00:00:00,Patient_3,1974-02-16,M
6,S5870,P004,HbA1c,6.1,%,2024-10-01 00:00:00,Patient_4,2017-08-22,F
7,S8208,P004,Glucose,9.1,mmol/L,2024-10-11 00:00:00,Patient_4,2017-08-22,F
8,S8154,P004,Creatinine,111.0,umol/L,2024-10-11 00:00:00,Patient_4,2017-08-22,F
9,S1059,P005,HbA1c,8.8,%,2024-10-09 00:00:00,Patient_5,1993-07-10,M


In [ ]:
# 2. TO show the Latest results
print("🔍 Patient Latest sample result")
query2 = """
WITH ranked AS (
  SELECT *,
    ROW_NUMBER() OVER (PARTITION BY patient_id ORDER BY collected_date DESC) AS rn
  FROM samples
)
SELECT *
FROM ranked
WHERE rn = 1;
"""
display(pd.read_sql(query2, conn))


🔍 Patient Latest sample result


,sample_id,patient_id,test_type,result_value,unit,collected_date,name,dob,gender,rn
0,S8743,P001,HbA1c,4.8,%,2024-10-07 00:00:00,Patient_1,2014-02-04,F,1
1,S6454,P002,HbA1c,5.4,%,2024-10-11 00:00:00,Patient_2,1987-04-13,M,1
2,S4902,P003,HbA1c,7.1,%,2024-10-11 00:00:00,Patient_3,1974-02-16,M,1
3,S8208,P004,Glucose,9.1,mmol/L,2024-10-11 00:00:00,Patient_4,2017-08-22,F,1
4,S1059,P005,HbA1c,8.8,%,2024-10-09 00:00:00,Patient_5,1993-07-10,M,1
5,S4959,P006,Glucose,6.6,mmol/L,2024-10-07 00:00:00,Patient_6,1973-08-21,M,1
6,S7457,P007,HbA1c,4.8,%,2024-10-02 00:00:00,Patient_7,1971-01-14,F,1
7,S2785,P008,Glucose,4.9,mmol/L,2024-10-10 00:00:00,Patient_8,1992-04-02,F,1
8,S3281,P009,HbA1c,5.9,%,2024-10-03 00:00:00,Patient_9,1951-06-29,F,1
9,S1989,P010,HbA1c,7.4,%,2024-10-11 00:00:00,Patient_10,1987-02-25,M,1


In [ ]:
Out
# 3. Calculate the gender average
print("🔍To show the gender avg results")
query3 = """
SELECT p.gender, s.test_type, AVG(s.result_value) AS avg_result
FROM sampleraw s
JOIN patientraw p ON s.patient_id = p.patient_id
GROUP BY p.gender, s.test_type;
"""
display(pd.read_sql(query3, conn))

🔍To show the gender avg results


,gender,test_type,avg_result
0,F,Creatinine,124.500000
1,F,Glucose,7.300000
2,F,HbA1c,5.340000
3,M,Cholesterol,7.600000
4,M,Creatinine,89.000000
5,M,Glucose,6.600000
6,M,HbA1c,6.557143


In [ ]:
# 4. Group by  Age group to analyis
print("🔍To show the gender avg results")
query4 = """
SELECT
  CASE
    WHEN strftime('%Y', 'now') - strftime('%Y', p.dob) < 40 THEN 'Under 40'
    WHEN strftime('%Y', 'now') - strftime('%Y', p.dob) < 65 THEN '40-64'
    ELSE '65+'
  END AS age_group,
  s.test_type,
  AVG(s.result_value) AS avg_result
FROM sampleraw s
JOIN patientraw p ON s.patient_id = p.patient_id
GROUP BY age_group, s.test_type;
"""
display(pd.read_sql(query4, conn))


🔍To show the gender avg results


,age_group,test_type,avg_result
0,40-64,Glucose,6.600000
1,40-64,HbA1c,5.950000
2,65+,HbA1c,5.900000
3,Under 40,Cholesterol,7.600000
4,Under 40,Creatinine,112.666667
5,Under 40,Glucose,7.300000
6,Under 40,HbA1c,6.088889
